In [ ]:
#| default_exp compiling

In [ ]:
#| export
import subprocess
import tensorflow as tf
import os
import pandas as pd

In [ ]:
#| export

def convert_model(train_ds):
    """Model conversion into TFLite model

    Args:
        train_ds (_dataset_): Training data used for the quantization process
    """

    model = 'models/keras_model'

       
    # Convert the model to the TensorFlow Lite format without quantization
    converter = tf.lite.TFLiteConverter.from_saved_model(model)
    model_no_quant_tflite = converter.convert()
    # Save the model to disk
    open('models/model_no_quant.tflite', "wb").write(model_no_quant_tflite)

    # Convert the model with quantization.
    converter = tf.lite.TFLiteConverter.from_saved_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    #uncomment to enable end-to-end int8-model, i.e. input and output is also int8/uint8.

    #converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    #converter.inference_input_type = tf.int8
    #converter.inference_output_type = tf.int8

    def representative_dataset():
        for data in tf.data.Dataset.from_tensor_slices((train_ds)).batch(1).take(100):
            yield [data[0]]

    #converter.representative_dataset = representative_dataset
    tflite_model = converter.convert()

    # Save the model.
    with open('models/model.tflite', 'wb') as f:
        f.write(tflite_model)

def convert_to_c_array():
    """C array conversion

    Args:
        model_file_name (string): TFLite model name for the conversion command
    """
    subprocess.run(['xxd -i models/model.tflite > models/model.cc'], shell=True)
    
    return True

def plot_size():
    """Plots the size difference before and after quantization

    Returns:
        pandas dataframe: Pandas dataframe containing information
    """


    size_no_quant_tflite = os.path.getsize('models/model_no_quant.tflite')
    size_tflite = os.path.getsize('models/model.tflite')
    
    frame = pd.DataFrame.from_records(
        [["TensorFlow Lite", f"{size_no_quant_tflite} bytes "],
        ["TensorFlow Lite Quantized", f"{size_tflite} bytes", f"{size_no_quant_tflite - size_tflite} bytes"]],
        columns = ["Model", "Size", "Size Reduced"], index="Model")

    return frame
